In [1]:
import gurobipy as gp
from gurobipy import Model, GRB

### Given:
## Periodic tasks with (peirod = deadline , execution time) or (period, execution time, deadline)
# T1 = (4, 1)
# T2 = (5, 2, 7)
# T3 = (20, 5)

# Hyperperiod = 20
# Frame Size = 4

In [2]:
hyperperiod = 20
frameSize = 4 

tasks = ['J1','J2','J3']
period = {'J1': 4,  # Task 1
          'J2': 5,  # Task 2
          'J3': 20, # Task 3
          }
execution_time = {"J1": 1, # Task 1
                  "J2": 2, # Task 2
                  "J3": 5, # Task 3
                  }
deadline = {'J1': 4,  # Task 1
            'J2': 7,  # Task 2
            'J3': 20, # Task 3
          }

In [3]:
# For each task, define the jobs that would run in the hyperperiod
jobs = []
for task in tasks:
    for i in range(1 , int(hyperperiod/period[task]) + 1):
        jobs.append(task+str(i))

# Define the frames that will run in one hyperperiod
frames = []
for i in range(1, int(hyperperiod/frameSize) + 1 ):
    frames.append("F" + str(i))

# define the release time of each job
releaseTimes = dict()
for job in jobs:
    releaseTimes[job] = period[job[0:2]]*(int(job[2]) - 1)

# define the deadline of each job
deadlines = dict()
for job in jobs:
    deadlines[job] = period[job[0:2]]*(int(job[2]) - 1) + deadline[job[0:2]]

# For each job, define the eligible frames
eligibleFrames = dict()
for job in jobs: 
    for i, frame in enumerate(frames):
        flag = True
        # Check that the job releases before or at the start of the frame
        if( releaseTimes[job]> frameSize * i):
            # print(period[job[0:2]]*(int(job[2]) - 1) , frameSize * i)
            flag = False

        # Check that the job ends after or equal the end of the frame
        if( deadlines[job] < frameSize * (i + 1)):
            flag = False
        eligibleFrames[(job,frame)] = flag


eligibleFrames_list = dict()
for job in jobs:
    eligibleFrames_list[job] = []
    for frame in frames:
        if eligibleFrames[(job,frame)]:
            eligibleFrames_list[job].append(frame)

eligibleJobs_list = dict()
for frame in frames:
    eligibleJobs_list[frame] = []
    for job in jobs:
        if eligibleFrames[(job,frame)]:
            eligibleJobs_list[frame].append(job)

In [4]:
# Create Gurobi model
m = Model()

# Disable presolve
m.setParam("Presolve", 0)

Set parameter Username
Set parameter LicenseID to value 2606738
Academic license - for non-commercial use only - expires 2026-01-06
Set parameter Presolve to value 0


In [5]:
###--- Variables ---###
# A Variable for each source to job edge
xSrcJ = m.addVars(jobs, vtype=GRB.CONTINUOUS, name=f"xSrcJ")

## For Each Job-To-Frame, define a variable:
xJF = dict()
for job in jobs:
    for frame in frames:
        if eligibleFrames[(job,frame)]:
            xJF[ (job,frame) ] = m.addVar(vtype=GRB.CONTINUOUS, name = f"x{job}{frame}")

## FOr each Frame-To-Sink, define a variable:
xFSink = m.addVars(frames, vtype=GRB.CONTINUOUS, name="xFSink")

In [6]:
###--- Constraints ---###

# Flow conservation for each job node:
for job in jobs:
    ## Both methods work:
    # m.addConstr(xSrcJ[job] == sum(xJF[(job, frame)] for frame in frames if eligibleFrames[(job, frame)]), name=f'cNode_{job}')
    m.addConstr(xSrcJ[job] == sum(xJF[(job, frame)] for frame in eligibleFrames_list[job]), name=f'cNode_{job}')
    

# Flow conservation for each frame node:
for frame in frames:
    # m.addConstr( sum( xJF[ (job,frame) ] for job in jobs if eligibleFrames[(job, frame)]) == xFSink[frame], name = f'cNode_{frame}' )
    m.addConstr( sum( xJF[ (job,frame) ] for job in eligibleJobs_list[frame]) == xFSink[frame], name = f'cNode_{frame}' )

# for each Source to Job flow, maximum flow must be less than or equal to job execution time: 
for job in jobs:
    m.addConstr( xSrcJ[job] <= execution_time[job[0:2]] )

# for each Frame to Sink flow, maximum flow must be less than or equal to frame size:
for frame in frames:
    m.addConstr( xFSink[frame] <= frameSize)

# All flows must be positive:
for job in jobs:
    m.addConstr(xSrcJ[job] >= 0) # Flow from source to job >= 0
    for frame in frames:
        if eligibleFrames[(job,frame)]:
            m.addConstr(xJF[(job,frame)] >= 0) # Flow from job to eligible Frame >= 0

for frame in frames:
        m.addConstr(xFSink[frame] >= 0) # Flow from Frame to Sink >= 0

In [7]:
###--- Objective ---###
# Maximize flow from frames to sink:

totalFlow = m.addVar(vtype=GRB.CONTINUOUS, name='TotalFlow')
m.addConstr( sum(xFSink[frame] for frame in frames) == totalFlow)

m.setObjective(totalFlow,GRB.MAXIMIZE)

In [8]:
# Optimize the model
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11370H @ 3.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
Presolve  0

Optimize a model with 60 rows, 30 columns and 93 nonzeros
Model fingerprint: 0xf4d49e8b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+30   2.000000e+30   1.000000e+00      0s
      16    1.8000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 16 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.800000000e+01


In [9]:
# Output the results
if m.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print("Flow From Source to Job:") 
    for job in jobs:
        print(f"S -> {job}: {xSrcJ[job].X}", end=" - ")
        # If all jobs's flow is equal to their execution time, then all jobs have been able to complete and the schedule is feasible
        unexecutedTime = xSrcJ[job].X - execution_time[job[0:2]]
        print(f"Flow - Execution time: {unexecutedTime}")
        if unexecutedTime > 0:
            print("Infeasible")
else:
    print("No optimal solution found.")

Optimal solution found!
Flow From Source to Job:
S -> J11: 1.0 - Flow - Execution time: 0.0
S -> J12: 1.0 - Flow - Execution time: 0.0
S -> J13: 1.0 - Flow - Execution time: 0.0
S -> J14: 1.0 - Flow - Execution time: 0.0
S -> J15: 1.0 - Flow - Execution time: 0.0
S -> J21: 2.0 - Flow - Execution time: 0.0
S -> J22: 2.0 - Flow - Execution time: 0.0
S -> J23: 2.0 - Flow - Execution time: 0.0
S -> J24: 2.0 - Flow - Execution time: 0.0
S -> J31: 5.0 - Flow - Execution time: 0.0


In [10]:
# Output the results
print("The Assignment of jobs in frames:")
if m.status == GRB.OPTIMAL:
    for job in jobs:
        for frame in eligibleFrames_list[job]:
            print(f"{job} -> {frame}: {xJF[(job,frame)].X}")
else:
    print("No optimal solution found.")

The Assignment of jobs in frames:
J11 -> F1: 1.0
J12 -> F2: 1.0
J13 -> F3: 1.0
J14 -> F4: 1.0
J15 -> F5: 1.0
J21 -> F1: 2.0
J22 -> F3: 2.0
J23 -> F4: 2.0
J24 -> F5: 2.0
J31 -> F1: 0.0
J31 -> F2: 2.0
J31 -> F3: 1.0
J31 -> F4: 1.0
J31 -> F5: 1.0


In [11]:
# Output the results
print("Flow From Frame to Sink")
if m.status == GRB.OPTIMAL:
    for frame in frames:
        print(f"{frame} -> Sink: {xFSink[(frame)].X}")
else:
    print("No optimal solution found.")

Flow From Frame to Sink
F1 -> Sink: 3.0
F2 -> Sink: 3.0
F3 -> Sink: 4.0
F4 -> Sink: 4.0
F5 -> Sink: 4.0
